In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from torch import nn
import numpy as np
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Dataset/sarcasmXsentiment/train-balanced-sarcasm.csv')

In [ ]:
df=df[['label', 'comment']]
df['comment'] = df['comment'].astype(str).fillna('')
df=df.iloc[:int(len(df)/2),:]
df=df.iloc[:5000,:]

#Using BERT Transformer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
inputs = tokenizer(df['comment'].to_list(), padding=True, truncation=True, return_tensors="pt", max_length=32)
inputs["labels"] = torch.tensor(df['label'])

In [ ]:
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
labels = inputs['labels']

train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks = train_test_split(
    input_ids, labels, attention_mask, test_size=0.3)

val_inputs, test_inputs, val_labels, test_labels, val_masks, test_masks = train_test_split(
    test_inputs, test_labels, test_masks, test_size=0.5)

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)

batch_size = 512
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001)
device = "cuda"
model = model.to(device)
model = torch.nn.DataParallel(model)

In [ ]:
for epoch in range(5):
    model.train()
    total_loss = 0

    with tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training", unit="batch") as pbar:
        for batch in pbar:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            avg_loss = total_loss / (pbar.n + 1)
            pbar.set_postfix({'loss': avg_loss})

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}")

    model.eval()
    val_loss = 0
    with torch.no_grad():
        with tqdm(val_dataloader, desc=f"Epoch {epoch+1} Validation", unit="batch") as pbar:
            for batch in pbar:
                input_ids, attention_mask, labels = batch
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()

                avg_val_loss = val_loss / (pbar.n + 1)
                pbar.set_postfix({'val_loss': avg_val_loss})

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch: {epoch+1}, Validation Loss: {avg_val_loss:.4f}")


Epoch 1 Training: 100%|██████████| 346/346 [14:26<00:00,  2.50s/batch, loss=0.587]


Epoch: 1, Training Loss: 0.5866


Epoch 1 Validation: 100%|██████████| 75/75 [01:08<00:00,  1.09batch/s, val_loss=0.54]


Epoch: 1, Validation Loss: 0.5399


Epoch 2 Training: 100%|██████████| 346/346 [14:26<00:00,  2.50s/batch, loss=0.519]


...

Epoch: 4, Validation Loss: 0.5109


Epoch 5 Training: 100%|██████████| 346/346 [14:26<00:00,  2.51s/batch, loss=0.426]


Epoch: 5, Training Loss: 0.4259


Epoch 5 Validation: 100%|██████████| 75/75 [01:08<00:00,  1.09batch/s, val_loss=0.534]

Epoch: 5, Validation Loss: 0.5337


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
model.eval()
test_loss = 0
predictions = []
true_labels = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits

        loss = outputs.loss
        test_loss += loss.item()

        preds = torch.argmax(logits, dim=-1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

avg_test_loss = test_loss / len(test_dataloader)
print(f"Test Loss: {avg_test_loss:.4f}")

predictions = np.array(predictions)
true_labels = np.array(true_labels)

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Loss: 0.5364
Accuracy: 0.7446
Precision: 0.7472
Recall: 0.7446
F1 Score: 0.7453


#Building Custom Neural Network

In [ ]:
word_list = df['comment'].str.split().explode().tolist()

word_list = list(set([word.lower() for word in word_list]))

word_list.append('<UNK>')

word_list.insert(0, '')

word_to_idx = {val: index for index, val in enumerate(word_list)}
idx_to_word = {index: val for index, val in enumerate(word_list)}

vocab_size = len(word_list)

In [ ]:
word_list[0]

''

In [ ]:
class GRUWithAttentionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(GRUWithAttentionModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.attention = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.embeddings(x)
        out, _ = self.gru(x)
        attn_weights = torch.nn.functional.softmax(self.attention(out).squeeze(2), dim=1)
        context = torch.sum(attn_weights.unsqueeze(2) * out, dim=1)
        fcc = self.fc(context)
        out = torch.sigmoid(fcc)

        return out

In [ ]:
def pad_or_truncate(tensor, max_length, padding_value=0):
    if len(tensor) > max_length:
        return tensor[:max_length]
    else:
        padding = (0, max_length - len(tensor))
        return F.pad(tensor, padding, value=padding_value)

In [ ]:
inputs=[torch.LongTensor([word_to_idx[w.lower()] for w in row['comment'].split()]) for _, row in df.iterrows()]
inputs = torch.stack([pad_or_truncate(tensor, 32) for tensor in inputs])

In [ ]:
inputs.shape

torch.Size([5000, 32])

In [ ]:
targets=torch.Tensor(df['label'].to_list())
targets.shape

torch.Size([5000])

In [ ]:
attention_model = GRUWithAttentionModel(vocab_size, 100, 128)
optimizer = torch.optim.AdamW(attention_model.parameters(), lr=0.00001)
criterion = nn.BCEWithLogitsLoss()
print("Model Instantiated")
epochs=5

Model Instantiated


In [ ]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    inputs, targets, test_size=0.3)

val_inputs, test_inputs, val_labels, test_labels = train_test_split(
    test_inputs, test_labels, test_size=0.5)

train_dataset = TensorDataset(train_inputs, train_labels)
val_dataset = TensorDataset(val_inputs, val_labels)
test_dataset = TensorDataset(test_inputs, test_labels)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for epoch in range(20):
    attention_model.train()
    total_loss = 0

    with tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training", unit="batch") as pbar:
        for batch in pbar:
            input_ids, labels = batch

            optimizer.zero_grad()

            outputs = attention_model(input_ids).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            avg_loss = total_loss / (pbar.n + 1)
            pbar.set_postfix({'loss': avg_loss})

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}")

    attention_model.eval()
    val_loss = 0
    with torch.no_grad():
        with tqdm(val_dataloader, desc=f"Epoch {epoch+1} Validation", unit="batch") as pbar:
            for batch in pbar:
                input_ids, labels = batch

                outputs = attention_model(input_ids).squeeze()
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                avg_val_loss = val_loss / (pbar.n + 1)
                pbar.set_postfix({'val_loss': avg_val_loss})

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch: {epoch+1}, Validation Loss: {avg_val_loss:.4f}")


Epoch 1 Training: 100%|██████████| 110/110 [00:06<00:00, 18.18batch/s, loss=0.72]


Epoch: 1, Training Loss: 0.7203


Epoch 1 Validation: 100%|██████████| 24/24 [00:00<00:00, 55.13batch/s, val_loss=0.819]


Epoch: 1, Validation Loss: 0.7164


Epoch 2 Training: 100%|██████████| 110/110 [00:05<00:00, 19.43batch/s, loss=0.715]


...

Epoch: 19, Validation Loss: 0.6978


Epoch 20 Training: 100%|██████████| 110/110 [00:05<00:00, 21.32batch/s, loss=0.703]


Epoch: 20, Training Loss: 0.6969


Epoch 20 Validation: 100%|██████████| 24/24 [00:00<00:00, 82.31batch/s, val_loss=0.881]

Epoch: 20, Validation Loss: 0.6973


In [ ]:
attention_model.eval()
test_loss = 0
predictions = []
true_labels = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, labels = batch

        outputs = attention_model(input_ids)

        loss = criterion(outputs.squeeze(), labels.float())
        test_loss += loss.item()

        probs = torch.sigmoid(outputs.squeeze())

        preds = (probs <= 0.5).float()

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8147
Precision: 0.6637
Recall: 0.8147
F1 Score: 0.7315


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
